# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv=pd.read_csv("output/cities.csv")
weather_csv

,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature(F),Humidity (%),Cloudiness,Wind Speed (MPH)
0,0,1,barrow,71.2906,-156.7887,17.62,73,20,10.36
1,1,2,talnakh,69.4865,88.3972,-2.79,99,89,4.94
2,2,3,new norfolk,-42.7826,147.0587,52.77,83,84,1.72
3,3,4,wanning,18.8003,110.3967,82.00,84,15,12.84
4,4,5,bredasdorp,-34.5322,20.0403,56.53,75,73,5.10
...,...,...,...,...,...,...,...,...,...
544,544,586,billings,45.7833,-108.5007,52.95,36,75,7.00
545,545,588,waddan,29.1614,16.1390,65.77,39,25,7.58
546,546,589,novyy urengoy,66.0833,76.6333,6.37,97,98,5.32
547,547,590,tres lagoas,-20.7511,-51.6783,76.66,53,29,8.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [8]:
locats = weather_csv[["Latitude","Longitude"]]
weight = weather_csv["Humidity (%)"]
fig_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(map_type="TERRAIN", layout = fig_layout)

fig.add_layer(gmaps.heatmap_layer(locats, weights = weight))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#I want snow!
good_cities=weather_csv.loc[weather_csv["Humidity (%)"]>80]
good_cities=good_cities.loc[weather_csv["Temperature(F)"]<32]
good_cities=good_cities.loc[weather_csv["Cloudiness"]>90]
good_cities=good_cities.loc[weather_csv["Wind Speed (MPH)"]<3]
good_cities

,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature(F),Humidity (%),Cloudiness,Wind Speed (MPH)
12,12,15,tiksi,71.6872,128.8694,4.12,96,98,2.42
58,58,64,pevek,69.7008,170.3133,12.22,91,92,1.99
186,186,199,qasigiannguit,68.8193,-51.1922,26.85,96,99,1.72
236,236,255,batagay-alyta,67.8006,130.4114,12.65,86,100,1.86
237,237,257,ilulissat,69.2167,-51.1000,26.62,93,100,0.00
301,301,326,saskylakh,71.9167,114.0833,6.30,97,96,1.74
310,310,335,chernyshevskiy,63.0128,112.4714,19.56,86,100,2.01
388,388,421,batagay,67.6333,134.6333,17.37,88,100,2.86
521,521,562,kautokeino,69.0125,23.0412,27.30,98,100,1.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
# geocoordinates
target_coordinates = "71.6872, 128.8694"
target_search = "Hotel"
target_radius = 5000


# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [75]:
hotel=[]
#print(good_cities["Latitude"].values[0])
for city in range(len(good_cities["City Name"])):
    try:
        # geocoordinates
        #target_coordinates="71.6872", "128.8694"
        lat=good_cities["Latitude"].values[city]
        long=good_cities["Longitude"].values[city]
        target_coordinates = f"{lat},{long}"
        target_search = "Hotel"
        target_radius = 5000
        #print(target_coordinates)


        # set up a parameters dictionary
        params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
        }

# base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
        response = requests.get(base_url, params=params)
        response_json=response.json()
        hotel.append(response_json["results"][0]["name"])
        #print(response_json)
    except:
        print(response_json)
        hotel.append("no hotel")
print(hotel)

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
['Gostinitsa', "Munitsipal'naya Gostinitsa", 'Hotel Diskobay', 'no hotel', 'Best Western Plus Hotel Ilulissat', 'no hotel', 'no hotel', 'no hotel', 'Thon Hotel Kautokeino']


In [77]:
good_cities["Hotel Name"]=hotel
good_cities

,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature(F),Humidity (%),Cloudiness,Wind Speed (MPH),Hotel Name
12,12,15,tiksi,71.6872,128.8694,4.12,96,98,2.42,Gostinitsa
58,58,64,pevek,69.7008,170.3133,12.22,91,92,1.99,Munitsipal'naya Gostinitsa
186,186,199,qasigiannguit,68.8193,-51.1922,26.85,96,99,1.72,Hotel Diskobay
236,236,255,batagay-alyta,67.8006,130.4114,12.65,86,100,1.86,no hotel
237,237,257,ilulissat,69.2167,-51.1000,26.62,93,100,0.00,Best Western Plus Hotel Ilulissat
301,301,326,saskylakh,71.9167,114.0833,6.30,97,96,1.74,no hotel
310,310,335,chernyshevskiy,63.0128,112.4714,19.56,86,100,2.01,no hotel
388,388,421,batagay,67.6333,134.6333,17.37,88,100,2.86,no hotel
521,521,562,kautokeino,69.0125,23.0412,27.30,98,100,1.10,Thon Hotel Kautokeino


In [78]:
#response.json()

In [79]:
#response_json["results"][0]["name"]


In [80]:
hotel_df=good_cities

In [86]:
hotel_df = hotel_df.rename(columns={"City Name": "City","Latitude":"Lat", "Longitude":"Lon"})

In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [90]:
# Add marker layer ontop of heat map
hotel_names=hotel_df["Hotel Name"]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"{Hotel}" for Hotel in hotel_names])
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))